<a href="https://colab.research.google.com/github/klysman08/Keggle/blob/master/Previs%C3%A3o_de_invent%C3%A1rio_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Módulos utilizados
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD


In [16]:
# Leitura dos arquivos, preenchendo valores faltantes com -1
train = pd.read_csv('train.csv', index_col='sku').fillna(-1)
test = pd.read_csv('test.csv', index_col='sku').fillna(-1)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
train.subtract

<bound method _arith_method_FRAME.<locals>.f of          national_inv  lead_time  ...  rev_stop  isBackorder
sku                               ...                       
1987420           5.0        2.0  ...         0            0
2171592          17.0        8.0  ...         0            0
3135863          11.0       10.0  ...         0            0
3108843           6.0        2.0  ...         0            0
2978127         180.0        8.0  ...         0            0
...               ...        ...  ...       ...          ...
1971041          62.0       -1.0  ...         0            0
3331163         784.0        2.0  ...         0            0
3403234          26.0       -1.0  ...         0            0
1926569           5.0       -1.0  ...         0            0
1905173         148.0        9.0  ...         0            0

[1350955 rows x 22 columns]>

In [0]:
# Separação de atributos de entrada (X) e saída (y)
X_train, y_train = train.drop('isBackorder', axis=1), train['isBackorder']

In [19]:
print(X_train.shape)
print(y_train.shape)
print(test)

(1350955, 21)
(1350955,)
         national_inv  lead_time  ...  stop_auto_buy  rev_stop
sku                               ...                         
3464610          46.0        8.0  ...              1         0
2949993         248.0        4.0  ...              0         0
1335649           3.0        4.0  ...              1         0
3178597          13.0        2.0  ...              1         0
1885965          46.0        8.0  ...              1         0
...               ...        ...  ...            ...       ...
1184635         230.0        2.0  ...              1         0
1937512           5.0        8.0  ...              1         0
1658698           9.0       -1.0  ...              1         0
1365873           9.0        2.0  ...              1         0
1225032           3.0        4.0  ...              1         0

[578982 rows x 21 columns]


In [20]:
# Ajuste do modelo de sequencial
model = Sequential()


model = Sequential()
model.add(Dense(64, input_dim=21, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=5,
          batch_size=128)

Epoch 1/5
1350955/1350955 [==============================] - 56s 41us/step - loss: 0.0578 - acc: 0.9921
Epoch 2/5
1350955/1350955 [==============================] - 55s 41us/step - loss: 0.0524 - acc: 0.9923
Epoch 3/5
1350955/1350955 [==============================] - 55s 41us/step - loss: 0.0536 - acc: 0.9922
Epoch 4/5
1350955/1350955 [==============================] - 56s 41us/step - loss: 0.0553 - acc: 0.9922
Epoch 5/5
1350955/1350955 [==============================] - 56s 42us/step - loss: 0.0568 - acc: 0.9922


In [21]:
model.summary()
#score = model.evaluate(train, test, batch_size=128)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                1408      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [22]:
#Predição da probabilidade de falta (y) para novos valores
y_pred = model.predict_proba(test)[:]

print(y_pred.shape)

(578982, 1)


In [0]:
#Cria e salva arquivo para submissão
test['isBackorder'] = y_pred
pred = test['isBackorder'].reset_index()
pred.to_csv('submissionKERASv1-2.csv',index=False)